In [1]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col, monotonically_increasing_id, lpad, to_date, concat, lit

In [2]:
# === STEP 2A: READ BRONZE DATA ===
print("Reading flights bronze data...")

flights_bronze_df = spark.read.table("unikargo_dev.01_bronze.unikargo_flights_bronze")

print(f"Raw flights count: {flights_bronze_df.count():,}")

Reading flights bronze data...
Raw flights count: 5,819,079


In [3]:
# === STEP 2B: DATA CLEANSING (flights_clean) ===
print("Cleansing flight data...")

flights_clean = (flights_bronze_df
               # Filter out invalid flights   
                .filter(col("flight_number").isNotNull())
                .filter(col("origin_airport").isNotNull())
                .filter(col("destination_airport").isNotNull())
                .filter(col("origin_airport") != F.col("destination_airport"))

                # Create flight_date as DateType
                .withColumn(
                "flight_date",
                to_date(
                    concat(
                    col("year").cast("string"),
                    lit("-"),
                    lpad(col("month").cast("string"), 2, "0"),
                    lit("-"),
                    lpad(col("day").cast("string"), 2, "0")
                    ),
                    "yyyy-MM-dd"
                ).cast("date")  # Explicit cast
    )
    # Remove rows with null flight_date
    .filter(col("flight_date").isNotNull())

    # Drop duplicates based on flight and route
    .dropDuplicates(["flight_number", "flight_date", "origin_airport", "destination_airport"]))

clean_count = flights_clean.count()
print(f"Clean flights count: {clean_count:,}")
print(f"Filtered out: {flights_bronze_df.count() - clean_count:,} records")

Cleansing flight data...
Clean flights count: 5,814,254
Filtered out: 4,825 records


In [4]:
flights_clean.show(2)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+-----------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|            metadata|flight_date|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+--------

In [5]:
# === STEP 2C: LOAD DIMENSION TABLES ===

print("Reading silver dimension tables...")

dim_date_silver_df = spark.read.table("unikargo_dev.02_silver.unikargo_dim_date_silver")
dim_airline_silver_df = spark.read.table("unikargo_dev.02_silver.unikargo_dim_airline_silver")
dim_airport_silver_df = spark.read.table("unikargo_dev.02_silver.unikargo_dim_airport_silver")

print(f"Dimensions loaded - Airlines: {dim_airline_silver_df.count()}, "
      f"Airports: {dim_airport_silver_df.count()}, "
      f"Dates: {dim_date_silver_df.count()}")

Reading silver dimension tables...
Dimensions loaded - Airlines: 14, Airports: 322, Dates: 5844


In [6]:
dim_date_silver_df.show(2)

+-------+---+--------+-----------+----------+----------+-----+----------+-------+----+
|date_sk|day|day_name|day_of_week| full_date|is_weekend|month|month_name|quarter|year|
+-------+---+--------+-----------+----------+----------+-----+----------+-------+----+
|   2923|  1|  Sunday|          7|2023-01-01|         1|    1|   January|      1|2023|
|   2924|  2|  Monday|          1|2023-01-02|         0|    1|   January|      1|2023|
+-------+---+--------+-----------+----------+----------+-----+----------+-------+----+
only showing top 2 rows


In [7]:
dim_date_silver_df.printSchema()

root
 |-- date_sk: long (nullable = true)
 |-- day: long (nullable = true)
 |-- day_name: string (nullable = true)
 |-- day_of_week: long (nullable = true)
 |-- full_date: date (nullable = true)
 |-- is_weekend: long (nullable = true)
 |-- month: long (nullable = true)
 |-- month_name: string (nullable = true)
 |-- quarter: long (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
# # join flights with dim_date on YEAR, MONTH, DAY, DAY_OF_WEEK
# flights_with_date_id = (
#     dim_date_silver_df.alias("f")
#     .join(
#         dim_date_silver_df.alias("d"),
#         on=["year", "month", "day", "day_of_week"], 
#         how="left"
#     )
#     # replace raw date columns with date_id
#     .drop("year", "month", "day", "day_of_week")
#     .withColumnRenamed("date_id", "date_id")
# )

# flights_with_date_id.show(5)

In [8]:
# === STEP 2D: CREATE FACT TABLE ===
print("Creating fact table with surrogate key joins...")

# Start with flights_clean


Creating fact table with surrogate key joins...


In [ ]:
# fact_flight = flights_clean.withColumn("flight_date_str", col("flight_date").cast("string"))
# dim_date_silver_df = dim_date_silver_df.withColumn("full_date_str", col("full_date").cast("string"))

# fact_flight_joined = fact_flight.join(
#     dim_date_silver_df.select("date_sk", "full_date_str", "day_of_week", "day_name"),
#     fact_flight["flight_date_str"] == dim_date_silver_df["full_date_str"],
#     "inner"
# )


# # Join with date dimension (preserve day_of_week for validation)
# fact_flight = flights_clean.join(
#     dim_date_silver_df.select("date_sk", "full_date", "day_of_week", "day_name"),
#     # fact_flight.flight_date == dim_date_silver_df.full_date,
#     col("flight_date_str") ==  col("full_date_str"),
#     "inner"
# )
# # .drop("flight_date", "full_date", "year", "month", "day")
# fact_flight.show(3)
# fact_flight.select("flight_date").distinct().show(5)
# dim_date_silver_df.select("full_date").distinct().show(5)


In [10]:
dim_date_silver_df.printSchema()
flights_clean.printSchema()

root
 |-- date_sk: long (nullable = true)
 |-- day: long (nullable = true)
 |-- day_name: string (nullable = true)
 |-- day_of_week: long (nullable = true)
 |-- full_date: date (nullable = true)
 |-- is_weekend: long (nullable = true)
 |-- month: long (nullable = true)
 |-- month_name: string (nullable = true)
 |-- quarter: long (nullable = true)
 |-- year: long (nullable = true)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: integer (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- origin_airport: string (nullable = true)
 |-- destination_airport: string (nullable = true)
 |-- scheduled_departure: integer (nullable = true)
 |-- departure_time: integer (nullable = true)
 |-- departure_delay: integer (nullable = true)
 |-- taxi_out: integer (nullable = true)
 |-- wheels_off: integer (nullable = true)


In [13]:
# Join with date dimension (preserve day_of_week for validation)

fact_flight = flights_clean.join(
    dim_date_silver_df.select("date_sk", "full_date", "day_of_week", "day_name"),
    flights_clean.flight_date == dim_date_silver_df.full_date,
    "inner"
).drop("flight_date", "full_date", "year", "month", "day")  # keep only surrogate key, not duplicate date cols

In [14]:
fact_flight.show(5)

+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+-------+-----------+--------+
|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|            metadata|date_sk|day_of_week|day_name|
+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+---

In [15]:
# Join airline dimension
fact_flight = fact_flight.join(
    dim_airline_silver_df.select("airline_sk", "iata_code"), 
    flights_clean.airline == dim_airline_silver_df.iata_code, 
    "inner"
    ).drop("airline", "iata_code")

In [16]:
# Join with origin airport dimension
origin_airport_dim = dim_airport_silver_df.select(
    col("airport_sk").alias("origin_airport_sk"),
    col("iata_code").alias("origin_iata")
)

fact_flight = fact_flight.join(
    origin_airport_dim,
    fact_flight.origin_airport == col("origin_iata"),
    "inner"
).drop("origin_airport", "origin_iata")

In [17]:
# Join with destination airport dimension
dest_airport_dim = dim_airport_silver_df.select(
    col("airport_sk").alias("destination_airport_sk"),
    col("iata_code").alias("dest_iata")
)

fact_flight = fact_flight.join(
    dest_airport_dim,
    fact_flight.destination_airport == col("dest_iata"),
    "inner"
).drop("destination_airport", "dest_iata")


In [18]:
fact_flight.show(5)

+-----------+-------------+-----------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+-------+-----------+--------+----------+-----------------+----------------------+
|day_of_week|flight_number|tail_number|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|            metadata|date_sk|day_of_week|day_name|airline_sk|origin_airport_sk|destination_airport_sk|
+-----------+-------------+-----------+-------------------+--------------+---------------+--------+----------+--------

In [21]:


# Add surrogate key for the fact table
fact_flight = fact_flight.withColumn("flight_sk", monotonically_increasing_id())

# Select and organize final columns
fact_flight_final = fact_flight.select(
    # Primary Key
    col("flight_sk"),
    
    # Foreign Keys (Surrogate Keys)
    col("date_sk"),
    col("airline_sk"),
    col("origin_airport_sk"),
    col("destination_airport_sk"),
    
    # Degenerate Dimensions
    col("flight_number"),
    col("tail_number"),
    
    # # Date attributes (from dimension)
    # col("day_of_week"),
    # col("day_name"),
    
    # Date attributes - Use qualified references from dimension table
    col("unikargo_dev.02_silver.unikargo_dim_date_silver.day_of_week"),
    col("unikargo_dev.02_silver.unikargo_dim_date_silver.day_name"),

    # Measures (converting to appropriate data types)
    col("scheduled_departure").cast("int"),
    col("departure_time").cast("int"),
    col("departure_delay").cast("double"),
    col("taxi_out").cast("double"),
    col("wheels_off").cast("int"),
    col("scheduled_time").cast("double"),
    col("elapsed_time").cast("double"),
    col("air_time").cast("double"),
    col("distance").cast("double"),
    col("wheels_on").cast("int"),
    col("taxi_in").cast("double"),
    col("scheduled_arrival").cast("int"),
    col("arrival_time").cast("int"),
    col("arrival_delay").cast("double"),
    col("diverted").cast("int"),
    col("cancelled").cast("int")
    # Note: Commented out columns that may not exist in all datasets
    # col("cancellation_reason"),
    # col("air_system_delay").cast("double"),
    # col("security_delay").cast("double"),
    # col("airline_delay").cast("double"),
    # col("late_aircraft_delay").cast("double"),
    # col("weather_delay").cast("double")
)

print(f"Fact table created with {fact_flight_final.count():,} records")

Fact table created with 5,328,614 records


In [22]:
# Show sample data
print("Sample fact table data:")
fact_flight_final.show(5, truncate=False)

Sample fact table data:
+---------+-------+----------+-----------------+----------------------+-------------+-----------+-----------+--------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+
|flight_sk|date_sk|airline_sk|origin_airport_sk|destination_airport_sk|flight_number|tail_number|day_of_week|day_name|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|
+---------+-------+----------+-----------------+----------------------+-------------+-----------+-----------+--------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+
|0        |1      |10 

In [23]:
# === STEP 2E: SAVE FACT TABLE ===
print("Saving fact table to silver layer...")

fact_flight_final.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("unikargo_dev.02_silver.unikargo_fact_flight_silver")

print("Flight fact table saved successfully!")

Saving fact table to silver layer...
Flight fact table saved successfully!


In [24]:
# === STEP 2F: DATA QUALITY CHECKS ===
print("\n=== DATA QUALITY CHECKS ===")

# Check day_of_week consistency (if original day_of_week exists in source)
print("Checking day of week consistency...")
if "day_of_week" in flights_bronze_df.columns:
    # Compare original vs calculated day_of_week
    day_check = flights_clean.withColumn(
        "calculated_day_of_week", 
        F.dayofweek("flight_date")  # 1=Sunday, 2=Monday, etc.
    ).withColumn(
        "original_day_of_week_adj",
        # Adjust original if it uses Monday=1 format vs Sunday=1
        F.when(col("day_of_week") == 7, 1)  # If original: Sunday=7, convert to 1
         .when(col("day_of_week") < 7, col("day_of_week") + 1)  # Shift others
         .otherwise(col("day_of_week"))
    ).filter(
        col("calculated_day_of_week") != col("original_day_of_week_adj")
    ).count()
    
    print(f"Day of week mismatches: {day_check}")

# Check for orphaned records (should be 0 after inner joins)
print("Checking data quality...")

# Verify all foreign keys exist
date_check = fact_flight_final.join(dim_date_silver_df, ["date_sk"], "left_anti").count()
airline_check = fact_flight_final.join(dim_airline_silver_df, ["airline_sk"], "left_anti").count()
origin_check = fact_flight_final.join(
    dim_airport_silver_df.select(col("airport_sk").alias("origin_airport_sk")), 
    ["origin_airport_sk"], "left_anti"
).count()
dest_check = fact_flight_final.join(
    dim_airport_silver_df.select(col("airport_sk").alias("destination_airport_sk")), 
    ["destination_airport_sk"], "left_anti"
).count()

print(f"Orphaned records check:")
print(f"  - Date orphans: {date_check}")
print(f"  - Airline orphans: {airline_check}")
print(f"  - Origin airport orphans: {origin_check}")
print(f"  - Destination airport orphans: {dest_check}")

# Basic statistics
print(f"\nFact table statistics:")
print(f"  - Total flights: {fact_flight_final.count():,}")
print(f"  - Date range: {fact_flight_final.count()} records")
print(f"  - Unique airlines: {fact_flight_final.select('airline_sk').distinct().count()}")
print(f"  - Unique origin airports: {fact_flight_final.select('origin_airport_sk').distinct().count()}")
print(f"  - Unique destination airports: {fact_flight_final.select('destination_airport_sk').distinct().count()}")

# Check for null values in key columns
null_checks = fact_flight_final.select([
    F.sum(col("flight_sk").isNull().cast("int")).alias("flight_sk_nulls"),
    F.sum(col("date_sk").isNull().cast("int")).alias("date_sk_nulls"),
    F.sum(col("airline_sk").isNull().cast("int")).alias("airline_sk_nulls"),
    F.sum(col("origin_airport_sk").isNull().cast("int")).alias("origin_airport_sk_nulls"),
    F.sum(col("destination_airport_sk").isNull().cast("int")).alias("destination_airport_sk_nulls")
]).collect()[0]

print(f"\nNull value checks:")
for field in null_checks.asDict():
    print(f"  - {field}: {null_checks[field]}")

print("\n=== PIPELINE COMPLETED SUCCESSFULLY ===")


=== DATA QUALITY CHECKS ===
Checking day of week consistency...
Day of week mismatches: 0
Checking data quality...
Orphaned records check:
  - Date orphans: 0
  - Airline orphans: 0
  - Origin airport orphans: 0
  - Destination airport orphans: 0

Fact table statistics:
  - Total flights: 5,328,614
  - Date range: 5328614 records
  - Unique airlines: 14
  - Unique origin airports: 322
  - Unique destination airports: 322

Null value checks:
  - flight_sk_nulls: 0
  - date_sk_nulls: 0
  - airline_sk_nulls: 0
  - origin_airport_sk_nulls: 0
  - destination_airport_sk_nulls: 0

=== PIPELINE COMPLETED SUCCESSFULLY ===


In [25]:
# # Check date ranges in both tables
# print("Date range in dim_date_silver_df:")
# dim_date_silver_df.select(F.min("full_date"), F.max("full_date")).show()

# print("Date range in flights_clean:")
# flights_clean.select(F.min("flight_date"), F.max("flight_date")).show()

In [26]:
# # Check for nulls
# print("Null count in dim_date_silver_df.full_date:")
# dim_date_silver_df.filter(F.col("full_date").isNull()).count()

# print("Null count in flights_clean.flight_date:")
# flights_clean.filter(F.col("flight_date").isNull()).count()

In [27]:
# Find overlapping dates
# overlapping_dates = flights_clean.select("flight_date").distinct() \
#     .join(dim_date_silver_df.select("full_date").distinct(), 
#           F.col("flight_date") == F.col("full_date"), "inner")
# overlapping_dates.count()